<a id='0.1'></a>
## 0.1 load required packages

[back to top](#top)

In [1]:
%run "..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

# library design specific tools
from ImageAnalysis3.library_tools import LibraryDesigner as ld
from ImageAnalysis3.library_tools import LibraryTools as lt
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

27300


<a id='0.2'></a>
## 0.2 required parameters for Genome

[back to top](#top)

In [2]:
# NEW mouse genome
genome_assembly = 'GRCm39'
reference_folder = os.path.join(r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse', f'{genome_assembly}_ensembl')
genome_folder = os.path.join(reference_folder, 'Genome')

In [3]:
class OTTable (dict):
    '''A python dictionary based implementation of the off-target table.'''
    def __missing__ (self, key):
        return 0

    def add_seq(self, seq, weight):
        '''Add a sequence to the OTTable with a given weight.'''
        self[seq] = self[seq] + weight

    def save_pkl(self, file_name):
        '''Save the OTTable as a pickle file.'''
        with open(file_name, 'wb') as f:
            pickle.dump(self, f, pickle.HIGHEST_PROTOCOL)
        print(f'Wrote the OTTable to {file_name}.')
    
    @staticmethod
    def load_pkl(file_name):
        '''Load a OTTable from a pickle file.'''
        with open(file_name, 'rb') as f:
            ottable = pickle.load(f)
        print(f'Load the OTTable from {file_name}.')
        return ottable

In [70]:
from seqint import seq2Int,seq2Int_rc
from tqdm import tqdm
import pickle
Default_Num_Threads=12

class countTable(dict):
    '''A python dictionary based implementation of the off-target table.'''
    def __init__(self, word_len=17, save_file=None, verbose=True):
        # internalize parameters
        self.word_len = word_len
        self.save_file = save_file
        self.verbose = verbose
        super().__init__()
        return
    def __missing__ (self, key):
        return 0
    # function to append sequence
    def add_seq(self, seq, weight):
        '''Add a sequence to the OTTable with a given weight.'''
        self[seq] = self[seq] + weight
    # function 
    def save_pkl(self, file_name=None):
        '''Save the OTTable as a pickle file.'''
        if file_name is None:
            file_name = self.save_file
        with open(file_name, 'wb') as f:
            pickle.dump(self, f, pickle.HIGHEST_PROTOCOL)
        print(f'Wrote the OTTable to {file_name}.')
    def load_sequences(
        self, sequences:list, word_len:int=0, weights:list=[],
        ):
        if self.verbose:
            print(f"Loading {len(sequences)} sequences into count_table.")
        if word_len <= 0 or word_len is None:
            word_len = self.word_len
            if self.verbose:
                print(f"- Use reference word length: {word_len}")
        else:
            if self.verbose:
                print(f"- Use given word length: {word_len}")
        # weights, if applicable
        # Use uniform weights if no weights are given
        if len(weights) == 0:
            weights = [1] * len(sequences)
        assert(len(weights) == len(sequences))
        # load
        for _seq, _w in zip(sequences, weights):
            # Find all K-mers and add to the OTTable
            for j in tqdm(range(len(_seq) - word_len + 1)): 
                self.add_seq(seq2Int(_seq[j:j+word_len].encode()), _w)
        
    @staticmethod
    def load_pkl(file_name):
        '''Load a OTTable from a pickle file.'''
        if file_name is None:
            file_name = self.save_file
        with open(file_name, 'rb') as f:
            ottable = pickle.load(f)
        print(f'Load the OTTable from {file_name}.')
        return ottable


In [37]:
# load all ref sequences
from ImageAnalysis3 import library_tools
seq_rd = library_tools.sequences.sequence_reader(genome_folder, auto_load_ref=True)
genome_seqs = [str(_r.seq) for _r in seq_rd.ref_seq_dict.values()]

-- load sequence: 1, size=195154279
-- load sequence: 10, size=130530862
-- load sequence: 11, size=121973369
-- load sequence: 12, size=120092757
-- load sequence: 13, size=120883175
-- load sequence: 14, size=125139656
-- load sequence: 15, size=104073951
-- load sequence: 16, size=98008968
-- load sequence: 17, size=95294699
-- load sequence: 18, size=90720763
-- load sequence: 19, size=61420004
-- load sequence: 2, size=181755017
-- load sequence: 3, size=159745316
-- load sequence: 4, size=156860686
-- load sequence: 5, size=151758149
-- load sequence: 6, size=149588044
-- load sequence: 7, size=144995196
-- load sequence: 8, size=130127694
-- load sequence: 9, size=124359700
-- load sequence: MT, size=16299
-- load sequence: X, size=169476592
-- load sequence: Y, size=91455967
-- load sequence: JH584299.1, size=953012
-- load sequence: GL456233.2, size=559103
-- load sequence: JH584301.1, size=259875
-- load sequence: GL456211.1, size=241735
-- load sequence: GL456221.1, size=206

In [71]:
genome_table = countTable()
#test_ct.load_sequences(['AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAACCCCCCCCCCCCCCCCCCCCCTTTTTTTTTTTTGGGGGGGGGGGGGGGG'])
genome_table.load_sequences(genome_seqs)

  0%|                                                                  | 107692/195154263 [00:00<03:01, 1076120.54it/s]

Loading 61 sequences into count_table.
- Use reference word length: 17


100%|██████████████████████████████████████████████████████████████████████████| 1960/1960 [00:00<00:00, 489626.91it/s]


In [72]:
genome_table = test_ct

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [73]:
genome_savefile = os.path.join(r'I:\References', f"{genome_assembly}_genome_table.pkl")

if os.path.isfile(genome_savefile):
    print(f"genome table file already exist, skip.")
else:
    print(f"Save genome table to file: {genome_savefile}")
    genome_table.save_pkl(genome_savefile)

Save genome table to file: I:\References\GRCm39_genome_table.pkl
Wrote the OTTable to I:\References\GRCm39_genome_table.pkl.


In [75]:
test_seq = genome_seqs[1][10000000:10000017]
test_seq

'ATATTTTATAAGAAATA'

In [76]:
%%timeit
_ref_table[seq2Int(test_seq.encode())]

423 ns ± 1.03 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
